In [2]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 2.9MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.8 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=3ed7d0e9e184719f228edb42bfc32014fe996fd7f7a5148457d34994dcf77d61
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle -v


Kaggle API 1.5.6


In [4]:
!kaggle competitions download -c classification-animal-classification


  0% 0.00/999 [00:00<?, ?B/s]
100% 999/999 [00:00<00:00, 723kB/s]


In [5]:
!unzip classification-animal-classification.zip

Archive:  classification-animal-classification.zip
  inflating: test_des.csv            
  inflating: test_summit_form.csv    
  inflating: train.csv               


In [7]:
import torch
import torch.optim as optim 
import torch.nn.functional as F 
# 소프트맥스 함수를 사용하기 위함. 
import numpy as np 
# numpy 
import pandas as pd
# pandas

torch.manual_seed(1)

data_zoo = pd.read_csv('train.csv')
data_zoo.head()
#print(len(data_zoo))

x_data = data_zoo.iloc[:,:-1]
#x_data.head()

y_data = data_zoo.iloc[:,[-1]]
#y_data.head()

print(x_data.shape)
print(y_data.shape)

x_train = torch.FloatTensor(x_data.values)
y_train = torch.LongTensor(y_data.values)

print(x_train.shape)
y_train = y_train.view(80)
print(y_train.shape)

(80, 16)
(80, 1)
torch.Size([80, 16])
torch.Size([80])


In [31]:
nb_class = 7
nb_data = len(y_train)

w = torch.zeros((16, nb_class),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

optimizer = optim.SGD([w,b], lr = 0.05)


nb_epochs = 200000

for epoch in range(nb_epochs+1):

    hypothesis = F.softmax(x_train.matmul(w)+b, dim=1) # 소프트맥스 함수 / 가설함수 

    y_one_hot = torch.zeros(nb_data, nb_class)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 10000 == 0:
        print('epoch {:4d}/{} cost : {:.6f}'.format(
            epoch, nb_epochs,cost.item()
        ))

epoch    0/200000 cost : 1.945910
epoch 10000/200000 cost : 1.284917
epoch 20000/200000 cost : 1.277221
epoch 30000/200000 cost : 1.275366
epoch 40000/200000 cost : 1.274523
epoch 50000/200000 cost : 1.274028
epoch 60000/200000 cost : 1.273694
epoch 70000/200000 cost : 1.273452
epoch 80000/200000 cost : 1.273266
epoch 90000/200000 cost : 1.273118
epoch 100000/200000 cost : 1.272996
epoch 110000/200000 cost : 1.272891
epoch 120000/200000 cost : 1.272794
epoch 130000/200000 cost : 1.272677
epoch 140000/200000 cost : 1.210975
epoch 150000/200000 cost : 1.170471
epoch 160000/200000 cost : 1.168599
epoch 170000/200000 cost : 1.167811
epoch 180000/200000 cost : 1.167355
epoch 190000/200000 cost : 1.167053
epoch 200000/200000 cost : 1.166837


In [32]:
hypothesis = F.softmax(x_train.matmul(w) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1)
#print(hypothesis)
#print(predict)
#print(y_train)

# 정확도 계산 
correct_prediction = predict.float() == y_train
#print(correct_prediction)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))


The model has an accuracy of 100.00% for the training set.


In [0]:
test = pd.read_csv('test_des.csv',header=None)
x_data = test.loc[:,0:15]
x_data = np.array(x_data)
x_test = torch.FloatTensor(x_data)
hypothesis = F.softmax(x_test.matmul(w)+b,dim=1)
predict = torch.argmax(hypothesis,dim=1)


In [21]:
predict

tensor([1, 3, 1, 0, 6, 3, 1, 5, 4, 1, 4, 3, 0, 0, 1, 0, 5, 0, 1, 1],
       grad_fn=<NotImplemented>)

In [0]:
predict = predict.detach().numpy().reshape(-1,1)
id = np.array([i for i in range(20)]).reshape(-1,1)
result = np.hstack([id,predict])
df = pd.DataFrame(result,columns=["Id","Category"])
df.to_csv("result.csv",index=False,header=True)


In [24]:
predict

array([[1],
       [3],
       [1],
       [0],
       [6],
       [3],
       [1],
       [5],
       [4],
       [1],
       [4],
       [3],
       [0],
       [0],
       [1],
       [0],
       [5],
       [0],
       [1],
       [1]])

In [25]:
result

array([[ 0,  1],
       [ 1,  3],
       [ 2,  1],
       [ 3,  0],
       [ 4,  6],
       [ 5,  3],
       [ 6,  1],
       [ 7,  5],
       [ 8,  4],
       [ 9,  1],
       [10,  4],
       [11,  3],
       [12,  0],
       [13,  0],
       [14,  1],
       [15,  0],
       [16,  5],
       [17,  0],
       [18,  1],
       [19,  1]])

In [26]:
!kaggle competitions submit -c classification-animal-classification -f result.csv -m "14010974_이기택"

100% 102/102 [00:00<00:00, 365B/s]
Successfully submitted to Classification : Animal Classification